# FHS data exploration

In [1]:
# import json
# from urllib.request import urlopen
# import requests
# import urllib.request
# import igraph as ig

# import chart_studio.plotly as py
# from plotly.offline import iplot
# import plotly.graph_objs as go
# from igraph import Graph

#Import the required packages
import networkx as nx 
import plotly.graph_objects as go
import pandas as pd

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pylab
import seaborn as sns
from scipy.stats import ttest_ind, mannwhitneyu, wilcoxon
import random

import networkx as nx

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

In [3]:
full_data = ''


network_data = True
if network_data:
    network_data = pd.read_sas(full_data + 'vr_sntwk_2008_m_0641s_v2_16.sas7bdat',encoding="unicode_escape",
                              format='sas7bdat')
    print ("network_data: loaded")
    print(network_data.shape)

apoe_data = False
if apoe_data:
    apoe_data = pd.read_sas(full_data + 'coh_off_apoe_16.sas7bdat',
                              format='sas7bdat')
    print ("apoe_data: loaded")
    print(apoe_data.shape)
    
neuro_data = False
if neuro_data:
    neuro_data = pd.read_sas(full_data + 'l_neurofact_2001_m_0546s_16.sas7bdat',
                              format='sas7bdat')
    print ("neuro_data: loaded")
    print(neuro_data.shape)
    
demvars_data = False
if demvars_data:
    demvars_data = pd.read_sas(full_data + 'demvars_for_170331_16.sas7bdat',
                              format='sas7bdat')
    print ("demvars_data: loaded")
    print(demvars_data.shape)

wkthru_data = False
if wkthru_data:
    wkthru_data = pd.read_sas(full_data + 'vr_wkthru_ex09_1_1001s_16.sas7bdat',
                              format='sas7bdat')
    print ("wkthru_data: loaded")
    print(wkthru_data.shape)



network_data: loaded
(193036, 33)


# Network data on wave 1

In [4]:
network_data.head()

,ALTERTYPE,CAUSEINIT,CAUSESEVERED,DISTMI1,DISTMI2,DISTMI3,DISTMI4,DISTMI5,DISTMI6,DISTMI7,DISTMI8,EGO_TREIMAN1,EGO_TREIMAN2,EGO_TREIMAN3,EGO_TREIMAN4,EGO_TREIMAN5,EGO_TREIMAN6,EGO_TREIMAN7,EGO_TREIMAN8,ALTER_TREIMAN1,ALTER_TREIMAN2,ALTER_TREIMAN3,ALTER_TREIMAN4,ALTER_TREIMAN5,ALTER_TREIMAN6,ALTER_TREIMAN7,ALTER_TREIMAN8,SPELLBEGIN,SPELLEND,idtype,alteridtype,idr,sharealterid
0,CHILD,NAMED,CENSORED,0.000000,0.000000,NaN,NaN,NaN,NaN,88.226804,88.533816,NaN,49.0,49.0,49.0,49.0,53.0,49.0,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,457.0,1.0,3.0,2.450817e+09,2.450788e+09
1,SPOUSE,NAMED,CENSORED,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,49.0,49.0,49.0,49.0,53.0,49.0,49.0,NaN,NaN,NaN,33.0,NaN,NaN,NaN,NaN,1.0,457.0,1.0,1.0,2.450817e+09,2.450617e+09
2,RELATIVENR,NAMED,CENSORED,2470.044117,2469.136853,2476.733168,2476.733168,2476.733168,537.573444,538.721584,538.721584,NaN,49.0,49.0,49.0,49.0,53.0,49.0,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,238.0,457.0,1.0,1.0,2.450817e+09,2.450896e+09
3,CHILD,ALTER DOB,CENSORED,NaN,0.000000,0.000000,0.000000,0.000000,NaN,13.705394,18.764749,NaN,53.0,52.0,52.0,52.0,57.0,53.0,57.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.0,457.0,1.0,3.0,2.450203e+09,2.450694e+09
4,CHILD,ALTER DOB,CENSORED,0.000000,0.000000,0.000000,0.000000,NaN,NaN,3.025986,15.116261,NaN,53.0,52.0,52.0,52.0,57.0,53.0,57.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,457.0,1.0,3.0,2.450203e+09,2.450760e+09


In [5]:
#########
# WAVE 1 -- select network data on wave 1 only
#########

df = network_data.copy()
w1 = df[(df['idtype']==1) & 
    ((df['alteridtype']==0)&(df['SPELLBEGIN']<29) | 
    (df['alteridtype']==1)&(df['SPELLBEGIN']<44)) &
    (df['SPELLEND']>44)]
print(df.shape)
print(w1.shape)

(193036, 33)
(44167, 33)


# Generate clean dataframe and extract network data

In [6]:
x = w1[[
    'idr',
    'sharealterid'
]].copy()

x.head()

,idr,sharealterid
1,2.450817e+09,2.450617e+09
13,2.450203e+09,2.450571e+09
16,2.450203e+09,2.450942e+09
17,2.450203e+09,2.450236e+09
18,2.450203e+09,2.450760e+09


In [7]:
# extract graph data from dataframe
G = nx.from_pandas_edgelist(x, 'idr', 'sharealterid')###, edge_attr=['Spouse'])

In [8]:
# Add network attributes for each node

#################################################
### -----NOTE Takes a long time to run -----
#################################################

x = pd.DataFrame(dict(

# DENSITY 
    # returns the density of the graph; d = 2m /(n(n-1))
    #### https://networkx.org/documentation/stable/reference/generated/networkx.classes.function.density.html
    DENSITY = nx.density(G), 
    
# CONSTRAINT
    # it is a measure of the extent to which a node v 
    # is invested in those nodes that are themselves invested in the neighbors of v
    #### https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.structuralholes.constraint.html
    CONSTRAINT = nx.constraint(G),  
        
# EFFECTIVE SIZE
    # based on the concept of redundancy
    # redundancy increases when ego's contacts are connected to each other as well
    # The nonredundant part of a person’s relationships it’s the effective size of her ego network
    #### https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.structuralholes.effective_size.html
    EFFECTIVE_SIZE = nx.effective_size(G),
    
# DEGREE
    # The node degree is the number of edges adjacent to the node
    #### https://networkx.org/documentation/stable/reference/classes/generated/networkx.Graph.degree.html
    DEGREE = dict(G.degree), # The node degree is the number of edges adjacent to the node

# DEGREE CENTRALITY
    # The degree centrality for a node v is the fraction of nodes it is connected to
    #### https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.degree_centrality.html
    DEGREE_CENTRALITY = nx.degree_centrality(G),
    
# EIGEN VECTOR CENTRALITY
    # Eigenvector centrality computes the centrality for a node based on the centrality of its neighbors. 
    #### https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.algorithms.centrality.eigenvector_centrality.html
    EIGENVECTOR = nx.eigenvector_centrality(G),
    
# KATZ CENTRALITY
    # Similar to Eigen Vector
    # It is a generalization of the eigenvector centrality
    #### https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.katz_centrality.html
    KATZ = nx.katz_centrality_numpy(G),
    
# CLOSENESS CENTRALITY
    # It is the reciprocal of the sum of the shortest path distances from u to all n-1 other nodes
    # Since the sum of distances depends on the number of nodes in the graph, 
    # closeness is normalized by the sum of minimum possible distances n-1
    CLOSENESS_CENTRALITY = nx.closeness_centrality(G),
    
# BETWENESS CENTRALITY
    # Betweenness centrality is the sum of the fraction of all-pairs shortest paths that pass through v:
    #### https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.algorithms.centrality.betweenness_centrality.html
    BETWEENNESS_CENTRALITY = nx.betweenness_centrality(G),
    
# CLUSTERING COEFFICIENT
    # is the fraction of possible triangles through that node that exist
    #### https://networkx.org/documentation/networkx-2.4/reference/algorithms/generated/networkx.algorithms.cluster.clustering.html
    CLUSTCOEF = nx.clustering(G),
)) 

In [9]:
x.head()

,DENSITY,CONSTRAINT,EFFECTIVE_SIZE,DEGREE,DEGREE_CENTRALITY,EIGENVECTOR,KATZ,CLOSENESS_CENTRALITY,BETWEENNESS_CENTRALITY,CLUSTCOEF
2.450817e+09,0.00087,1.000000,1.000000,1,0.000128,1.858149e-10,0.000087,0.098151,0.000000,0.000000
2.450617e+09,0.00087,0.506173,2.333333,3,0.000383,5.258004e-09,0.000071,0.110117,0.000231,0.333333
2.450203e+09,0.00087,0.107119,5.666667,21,0.002680,4.653956e-03,0.001595,0.194065,0.000304,0.766667
2.450571e+09,0.00087,0.075507,15.965517,29,0.003701,5.537345e-03,-0.010123,0.195987,0.001904,0.465517
2.450942e+09,0.00087,0.049222,37.928571,56,0.007147,2.650506e-02,-0.006316,0.211850,0.009554,0.328571


In [10]:
x = x.rename({'index':'idr'},axis=1)

In [11]:
x.to_csv('FHS_participants_extracted_network_features_wave1.csv')